In [23]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv
import os

In [21]:
loader = DirectoryLoader("/Users/hritik/Desktop/Grading Doc RAG/Document", glob="**/*.html",loader_cls=UnstructuredHTMLLoader)
docs = loader.load()
len(docs)

166

In [24]:
dotenv_path = '../.env'
load_dotenv(dotenv_path)
HUGGINGFACEHUB_API_TOKEN = os.getenv("HF_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HUGGINGFACEHUB_API_TOKEN, model_name="sentence-transformers/all-MiniLM-l6-v2"
)

In [25]:
db = Chroma.from_documents(docs, embeddings, persist_directory='vectordb/chroma/')

In [28]:
print(db._collection.count())

200
